% CGRtools Tutorial
% Dr. Ramil Nugmanov; Dr. Timur Madzhidov; Ravil Mukhametgaleev
% Mar 25, 2019

# CGRtools Tutorial

(c) 2019, Dr. Ramil Nugmanov; Dr. Timur Madzhidov; Ravil Mukhametgaleev

Installation instructions of CGRtools package information and tutorial's files see on `https://github.com/cimm-kzn/CGRtools`

NOTE: Tutorial should be performed sequentially from the start. Random cell running will lead to unexpected results. 

In [ ]:
import pkg_resources
if pkg_resources.get_distribution('CGRtools').version.split('.')[:2] != ['3', '1']:
    print('WARNING. Tutorial was tested on 3.1 version of CGRtools')
else:
    print('Welcome!')

In [ ]:
# load data for tutorial
from pickle import load
from traceback import format_exc

with open('molecules.dat', 'rb') as f:
    molecules = load(f) # list of MoleculeContainer objects
with open('reactions.dat', 'rb') as f:
    reactions = load(f) # list of ReactionContainer objects

m1, m2, m3, m4 = molecules # molecule
m7 = m3.copy()
m11 = m3.copy()
m11.standardize()
m12 = m3.copy()
m7.standardize()
r1 = reactions[0] # reaction
m5, m6 = r1.reactants[:2]
m8 = m7.substructure([4, 5, 6, 7, 8, 9], as_view=False)
m9 = m6.substructure([5, 6,7, 8], as_view=False) # acid
m10 =  r1.products[0].copy()

# 1. Data types and operations with them

CGRtools has subpackage *containers* with data structures classes:

* *MoleculeContainer* - for molecular structure
* *ReactionContainer* - for chemical reaction 
* *CGRContainer* - for Condensed Graph of Reaction
* *QueryContainer* - queries for substructure search in molecules
* *QueryCGRContainer* - queries for substructure search in CGRs

In [ ]:
from CGRtools.containers import * # import all containers

## 1.1. MoleculeContainer
Molecules are represented as undirected graphs. Molecules contain *Atom* objects and *Bond* objects.

*Atom* objects are represented as dictionary with unique number for each atom as key.  

*Bond* objects are stored as sparse matrix with adjacent atoms pair as keys for rows and columns.

Hereafter, *atom number* is unique integer used to enumerate atoms in molecule. Please, don't confuse it with element number in Periodic Table, hereafter called *element number*.

Methods for molecule handling and the arguments of *MoleculeContainer* are described below.

In [ ]:
m1.meta # dictionary for molecule properties storage. For example, DTYPE/DATUM fields of SDF file are read into this dictionary

In [ ]:
m1 # MoleculeContainer supports depiction and graphic representation in Jupyter notebooks.

In [ ]:
m1.depict() # depiction returns SVG image in format string

In [ ]:
with open('molecule.svg', 'w') as f: # saving image to SVG file
    f.write(m1.depict())

In [ ]:
m_copy = m1.copy() # isolated copy of molecule
m_copy

In [ ]:
len(m1) # get number of atoms in molecule
# or 
m1.atoms_count

In [ ]:
m1.bonds_count # number of bonds

In [ ]:
m1.atoms_numbers # list of atoms numbers

In [ ]:
# this method calculates additional atoms attributes: number of neighbors and hybridization. See below for usage
m1.reset_query_marks() # by default this attributes are None (for speed-up)
m3.reset_query_marks()

The following notations are used for hybridization of atoms. Values are given as numbers below (in parenthesis symbols that are used in SMILES-like signatures are shown):

* 1 (s) - all bonds of atom are single, i.e. sp3 hybridization
* 2 (d) - atom has one double bond and others are single, i.e. sp2 hybridization
* 3 (t) - atom has one triple or two double bonds and other are single, i.e. sp hybridization
* 4 (a) - atom is in aromatic ring

Neighbors and hybridizations atom attributes are required for **substructure operations** and structure standardization. See below

In [ ]:
# iterate over atoms using its numbers
list(m1.atoms())  # works the same as dict.items()

In [ ]:
# iterate over bonds using adjacent atoms numbers
list(m1.bonds())

In [ ]:
# access to atom by number
m1.atom(1)

In [ ]:
try:
    m1.atom(10) # raise error for absent atom numbers
except KeyError:
    print(format_exc())

In [ ]:
# access to bond using adjacent atoms numbers
m1.bond(1, 4)

In [ ]:
try:
    m1.bond(1, 3) # raise error for absent bond
except KeyError:
    print(format_exc())

#### Atom objects are dictinary-like classes which store information about:
* element
* isotope
* charge
* multiplicity
* xyz coordinates

Also atoms has methods for data integrity checks and include some internally used data.

In [ ]:
a = m1.atom(1)

# access to information
a.element # element symbol
# or
a['element']

In [ ]:
a.charge # formal charge
# or
a['charge']

In [ ]:
a.multiplicity # atom multiplicity. None if not set
# or
a['multiplicity']

In [ ]:
a.isotope # atom isotope. Default isotope if not set. Default isotopes are the same as used in InChI notation
# or
a['isotope']

In [ ]:
a.x # coordinates
a.y
a.z
# or
a['x']
a['y']
a['z']

In [ ]:
a.neighbors # Number of neighboring atoms is calculated by reset_query_marks method as shown above. It is read-only. 

In [ ]:
a.hybridization # Atoms hybridization is calculated by reset_query_marks method as shown above. It is read-only. 

In [ ]:
try:
    a.hybridization = 2 # Not subsettable. Read-only! Thus error is raised.
except AttributeError:
    print(format_exc())

#### Atomic attributes are subsettable.

CGRtools has integrity checks for verification of changes induced by user

In [ ]:
a.isotope = 16
# or 
a['isotope'] = 16

In [ ]:
m1.flush_cache() # due to caching used for speed-up one needs to reset cache of molecule to observe changes in atoms and bonds 
m1

In [ ]:
try:
    a.isotope = 0 # raise error. Isotope with mass 0 could not exist
except ValueError:
    print(format_exc())

In [ ]:
# bond objects also are dictionary-like classes which store information about bond order
b = m1.bond(3, 4)

b.order
#
b['order']

In [ ]:
b.order = 1 # order change also possible
# or
b['order'] = 1

In [ ]:
m1.flush_cache() #after flushing cashe one could see molecule with changes
m1

In [ ]:
try:
    b['order'] = 0 # error raises. Bond order 0 is invalid. To delete bond use method described below
except ValueError:
    print(format_exc())

One should to use `delete_bond` method to break bond

In [ ]:
m1.delete_bond(3, 4)
m1                    #Note that cashe flushing is not required here.

Method `delete_atom` removes atom from the molecule

In [ ]:
m1.delete_atom(3)
m1

In [ ]:
m_copy # copy unchanged!

*Atoms* and *bonds* objects can be converted into integer representation that could be used to classify their types.

*Atom* type is represented by 21 bit code rounded to 32 bit integer number:

* 7 bits stand for atom number (2 ** 7 - 1 == 127, currently 118 elements are presented in Periodic Table)
* 9 bits are used for isotope (511 posibilities, highest known isotope is ~300)
* 3 bits stand for formal charge. Charges range from -3 to +3 rescaled to range 0-6
* 2 bits are used for multiplicity.

In [ ]:
int(a)
# 131596 == 0001000 000010000 011 00
# 0001000 == 8 Oxygen
# 000010000 == 16 isotope
# 011 == 3 (3 - 3 = 0) uncharged
# 00 == 0 hasn't multiplicity

In [ ]:
int(b)  # bonds are encoded by their order

In [ ]:
print(m1.atom_implicit_h(1)) # get number of implicit hydrogens on atom 1
print(m1.atom_explicit_h(1)) # get number of explicit hydrogens on atom 1
print(m1.atom_total_h(1)) # get total number of hydrogens on atom 1

In [ ]:
m1

In [ ]:
m1.check_valence() # return list of numbers of atoms with invalid valences

In [ ]:
m4 # molecule with valence errors

In [ ]:
m4.check_valence() 

In [ ]:
m3

In [ ]:
m3.sssr # Method for application of Smallest Set of Smallest Rings algorithm for rings 
        # identification. Returns list of lists of atoms forming smallest rings

#### Connected components.
Sometimes molecules has disconnected components (salts etc).

One can find them and split molecule to separate components.

In [ ]:
m2 # it's a salt represented as one graph

In [ ]:
m2.connected_components # list of lists of atoms belonging to graph components

In [ ]:
anion, cation = m2.split() # split molecule to components

In [ ]:
anion # graph of only one salt component

In [ ]:
cation # graph of only one salt component

#### Union of molecules
Sometimes it is more convenient to represent salts as ion pair. Otherwise ambiguity could be introduced, for example in reaction of salt exchange:

**Ag+** + **NO3-** + **Na+** + **Br-** = **Ag+** + **Br-** + **Na+** + **NO3-**. Reactants and products sets are the same. 

In this case one can combine anion-cation pair into single graph. It could be convenient way to represent other molecule mixtures.

In [ ]:
salt = anion | cation 

# or 

anion.union(cation)

salt # this graph has disconnected components, it is considered single compound now

#### Substructures could be extracted from molecules.

By default, returned substructures are read-only projections of original molecule (except attributes of atoms/bonds). 

Changes in original molecule (bond breaking/formation, atom insertion/deletion, atom/bond attributes changes) will be mirrored in projection.  

Projections share the same neighbors and hybridization attributes as in initial molecule even if could be wrong for substructure 

In [ ]:
proj = m3.substructure([4,5,6,7,8,9])  # projection of substructure
proj

In [ ]:
m3.atom(4).neighbors 

In [ ]:
proj.atom(4).neighbors # same as in original molecule and not as it should be in substructure

In [ ]:
from networkx.exception import NetworkXError
try:
    proj.reset_query_marks() # change of structure for projections is blocked
except NetworkXError:
    print(format_exc())

In [ ]:
benzene = m3.substructure([4,5,6,7,8,9], as_view=False) # Substructure could be extracted as isolated graph (not projection)
benzene

In [ ]:
benzene.atom(4).neighbors is None # empty attribute. Substructure is a new molecule here. We need to call reset_query_marks

In [ ]:
benzene.reset_query_marks()

In [ ]:
benzene.atom(4).neighbors # now number of neighbors for atom 4 is 2. It is not 3 as above where projection was used.

Note:

* Projection of projection also projection of original molecule
* Projection can be converted to isolated molecule by calling method copy()

In [ ]:
proj_copy = proj.copy() # turning projection into molecule using "copy" method
proj_copy

In [ ]:
proj_copy.reset_query_marks() # This not a projection any more but a new molecule

Changes in projection are mirrored. See example:

In [ ]:
m3.delete_bond(4, 5) # we've deleted bond
m3

In [ ]:
proj.flush_cache() # remove cached image. Note that in projection the bond is also deleted.
proj

`augmented_substructure` is a substructure consisting from atoms and a given number of shells of neighboring atoms around it.
**deep** argument is a number of considered shells. 

It also returns projection by default.

In [ ]:
aug = m3.augmented_substructure([10], deep=2) #  atom 10 is Nitrogen
aug

In [ ]:
aug.atom(10).hybridization #atom has two incident double bond.

#### Atoms Ordering.
This functionality is used for canonic numbering of atoms in molecules. Prime number multiplication based Morgan algorithm is used for atom ranking. Property `atoms_order` returns dictionary of atom numbers as keys and their ranks according to canonicalization as values. Equal rank mean that atoms are symmetric (are mapped to each other in automorhisms). In present version, instead of sequential ranks prime numbers are returned.


In [ ]:
m5.atoms_order

#### Atom number can be changed by `remap` method.

This method is useful when it is needed to change order of atoms in molecules. First argument to `remap` method is dictionary with existing atom numbers as keys and desired atom number as values. It is possible to change atom numbers for only part of atoms. Atom numbers could be non-sequencial but need to be unique. 

If argument *copy* is set ***True*** new object will be created, else existing molecule will be changed. Default is ***False***. 

In [ ]:
for n, a in m5.atoms():
    print(n, a.element)
for n, m, b in m5.bonds():
    print(m, n, b.order)

In [ ]:
remapped = m5.remap({4:2}, copy=True)
remapped

In [ ]:
for n, a in remapped.atoms():
    print(n, a.element)
for n, m, b in remapped.bonds():
    print(m, n, b.order)

## 1.2. ReactionContainer

*ReactionContainer* objects has the following properties:

* **reactants** - list of reactants molecules
* **reagents** - list of reagents molecules
* **products** - list of products molecules
* **meta** - dictinary of reaction metadata (DTYPE/DATUM block in RDF)

In [ ]:
r1 # depiction supported

In [ ]:
r1.meta

In [ ]:
print(r1.reactants, r1.products)  # Access to lists of reactant and products. Molecules' signatures are returned by print() method.
reactant1, reactant2, reactant3 = r1.reactants
product = r1.products[0]

Reactions also has `standardize`, `aromatize`, `reset_query_marks`, `implicify_hydrogens` and `explicify_hydrogens` methods (see part 3). These methods are applied independently to every molecule in reaction.

## 1.3. CGR
*CGRContainer* object is similar to *MoleculeConrtainer*, except some methods. The following methods are not suppoted for *CGRContainer*:

* aromatize
* standardize
* implicify_hydrogens
* explicify_hydrogens
* atom_implicit_h
* atom_explicit_h
* atom_total_h
* check_valence

*CGRContainer* also has some methods absent in *MoleculeConrtainer*:

* centers_list
* center_atoms
* center_bonds

*CGRContainer* is undirected graph. Atoms and bonds in CGR has two states: reactant and product.

#### Composing to CGR

As mentioned above, atoms in *MoleculeContainer* have unique numbers. These numbers are used as atom-to-atom mapping in CGRtools upon CGR creation. Thus, atom order for molecules in reaction should correspond to atom-to-atom mapping.  

Pair of molecules can be transformed into CGR. Notice that, the same atom numbers in reagents and products imply the same atoms.

Reaction also can be composed into CGR. Atom numbers of molecules in reaction are used as atom-to-atom mapping of reactants to products.

In [ ]:
cgr1 = m7 ^ m8 # CGR from molecules
# or 
m7.compose(m8)

print(cgr1)  # CGRcontainer object currently doesn't support depiction. CGR signature is printed out.

This is CGR (depiction is made by ChemAxon externally). You can see changed bonds connected to ring.
![cgr1.png](cgr1.png)

In [ ]:
r1

In [ ]:
cgr2 = ~r1 # CGR from reactions

# or 

r1.compose()
print(cgr2) # signature is printed out. 

It is CGR for reaction (depiction is made by ChemAxon externally).
![cgr2.png](cgr2.png)

In [ ]:
cgr2.reset_query_marks() # CGRs also has reset_query_marks method

In [ ]:
a = cgr2.atom(2) # atom access is the same as for MoleculeContainer

In [ ]:
a.element # element attribute

# or 

a['element']

In [ ]:
a.isotope # isotope attribute

# or 

a['isotope']

For *CGRContainer* attributes `charge`, `multiplicity`, `x`, `y`, `z`, `neighbors` and `hybridization` refer to atom state in reactant of reaction;  arguments `p_charge`, `p_multiplicity`, `p_x`, `p_y`, `p_z`, `p_neighbors` and `p_hybridization` could be used to extract atom state in product part in reaction.

In [ ]:
a.charge # charge of atom in reactant

# or 

a['charge']

In [ ]:
a.p_charge # charge of atom in product

#or 

a['p_charge']

In [ ]:
a.p_multiplicity # multiplicity of atom in product. It is None and thus not returned

In [ ]:
a.p_x  # coordinates of atom in product
a.p_y
a.p_z

In [ ]:
a.neighbors # number of neighbors of atom in reactant

In [ ]:
a.p_neighbors # number of neighbors of atom in product

In [ ]:
a.hybridization # hybridization of atom in reactant. 1 means only single bonds are incident to atom

In [ ]:
a.p_hybridization # hybridization of atom in product. 1 means only single bonds are incident to atom

In [ ]:
b = cgr1.bond(4, 10) # take bond

#### Bonds has `order` and `p_order` attribute
If `order` attribute value is ***None***, it means that bond was formed  
If `p_order` is ***None***, it means that bond was broken  

Both `order` and `p_order` can't be ***None***

In [ ]:
b.order # bond order in reactant

In [ ]:
b.p_order is None # bond order in product in None

**CGR can be decomposed back to reaction**, i.e. reactants and products.

Notice that CGR can lose information in case of unbalanced reactions (where some atoms of reactant does not have counterpart in product, and vice versa). Decomposition of CGRs for unbalanced reactions back to reaction may lead to strange (and erroneous) structures.

In [ ]:
reactant_part, product_part = ~cgr1 # CGR of unbalanced reaction is decomposed back into reaction
# or 
cgr1.decompose() 

In [ ]:
reactant_part # reactants extracted. One can notice it is initial molecule

In [ ]:
product_part #extracted products. Originally benzene was the product.

For decomposition of *CGRContainer* back into *ReactionContainer* `CGRpreparer` class can be used. `CGRpreparer` is callable object

In [ ]:
from CGRtools import CGRpreparer # import of CGRpreparer
preparer = CGRpreparer()         # initialization of CGRpreparer

In [ ]:
decomposed = preparer.decompose(cgr2) # decomposition of CGR2 into reaction

In [ ]:
decomposed # You can see that water absent in products initially was restored. 
# This is a side-effect of CGR decomposing that could help with reaction balancing. 
# But balancing using CGR decomposition works correctly only if minor part atoms are lost 
# but multiplicity and formal charge are saved. In next release electronic state balansing will be added.

In [ ]:
r1 # compare with initial reaction

## 1.4 Queries

CGRtools supports special objects for Queries. Queries are designed for substructure isomorphism. User can set number of neighbors and hybridization by himself (in molecules they could be calculated but could not be changed).

Queries don't have `reset_query_marks` method

In [ ]:
from CGRtools.containers import*

In [ ]:
m10.reset_query_marks()
m10 # ether

In [ ]:
carb = m10.substructure([5,7,8, 2]) # extract projection of carboxyl fragment
carb

In [ ]:
q = QueryContainer(carb)  # convert fragment into query
print(q) # QueryContainer don't support depiction yet but signatures can be extracted

CGRs also can be transformed into Query.

`QueryCGRContainer` is similar to QueryContainer class for CGRs and has the same API.

`QueryCGRContainer` take into account state of atoms and bonds in reactant and product, including neighbors and hybridization

In [ ]:
cgr1.reset_query_marks()        # cgr1 is CGRContaier its labels could be calculated 
cgr_q = QueryCGRContainer(cgr1) # transfrom CGRContainer into QueryCGRContainer
print(cgr_q)                    # print out signature of query

## 1.5. Molecules, CGRs, Reactions construction

CGRtools has API for objects construction from scratch.

CGR and Molecule has methods `add_atom` and `add_bond` for adding atoms and bonds.

In [ ]:
from CGRtools.containers import *

In [ ]:
m = MoleculeContainer() # new empty molecule

m.add_atom('C')  # add Carbon atom using element symbol
m.add_atom(6)    # add Carbon atom using element number. {'element': 6} is not valid, but {'element': 'O'} is also acceptable
m.add_atom({'element': 'O', 'charge': -1}) # add negatively charged Oxygen atom. Similarly other atomic properties can be set

# add_atom has second argument for setting atom number. 
# If not set, the next integer after the biggest among already created will be used.
m.add_atom({'element': 'Na', 'charge': 1}, 4)

In [ ]:
m.add_bond(1, 2, 1) # add bond with order = 1 between atoms 1 and 2
m.add_bond(3, 2, {'order': 1}) # the other possibility to set bond order

In [ ]:
m.calculate2d() #experimental function to calculate atom coordinates. Has number of flaws yet
m

Reactions can be constructed from molecules

In [ ]:
r = ReactionContainer() # empty reaction
r.reactants.append(m1) # add reactant
r.products.append(m11)  # add product
# or
r = ReactionContainer(reactants=[m1], products=[m11]) # one-step way to construct reaction
# or
r = ReactionContainer([m1], [m11]) # first list of MoleculeContainers is interpreted as reactants, second one - as products

In [ ]:
r

In [ ]:
# reactants, products, reagents attributes are list-like. 
r.products.append(m.copy()) # One can add (or remove) molecules directly to this list of products returned by r.products
r.flush_cache()

In [ ]:
r # coordinates will left unchanged. Thus depiction could look wrong. 

In [ ]:
r.fix_positions() # this method fixes coordinates of molecules in reaction
r

*QueryContainers* can be constructed in the same way as *MoleculeContainers*.

Unlike other containers *QueryContainers* additionally support atoms, neighbors and hybridization lists.

In [ ]:
q = QueryContainer() # creation of empty container
q.add_atom({'element': ['N', 'O']}) # add atom list: N or O atom, any isotope and multiplicity, neutral, 
                                    # number of neighbors and hybridization are irrelevant
q.add_atom({'element': 'C', 'neighbors': [2, 3]}) # add carbon atom, any isotope and multiplicity, neutral, 
                                                  # has 2 or 3 non-hydrogen heighbors
q.add_atom({'element': 'O'}) # add oxygen atom, any isotope and multiplicity, neutral, 
                             # number of neighbors and hybridization are irrelevant 
q.add_bond(1, 2, 1) # add single bond between atom 1 and 2 
q.add_bond(2, 3, 2) # add double bond between atom 1 and 2 
# any amide or carboxilic group will fit this query

In [ ]:
print(q) # print out signature (SMILES-like)

## 1.6. Extending CGRtools

You can easily customize CGRtools for your tasks.  
CGRtools is OOP-oriented library with subclassing and inheritance support.

As an example, we show how special marks on atoms for ligand donor centers can be added.

In [ ]:
from CGRtools.containers import MoleculeContainer
from CGRtools.attributes import Atom

In [ ]:
class MarkedAtom(Atom):  # this class will inherite Atom class
    __slots__ = '__mark' # all new attributes should be slotted!
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.__mark = None # set default value for added attribute

    @property
    def mark(self):        # created new property 
        return self.__mark
    
    @mark.setter
    def mark(self, mark):
        # do some checks and calculations
        self.__mark = mark


In [ ]:
class MarkedMoleculeContainer(MoleculeContainer): # MarkedAtomContainer inherits MoleculeContainer
    node_attr_dict_factory = MarkedAtom # override atom container

In [ ]:
m = MarkedMoleculeContainer() # create newly developed container MarkedMoleculeContainer
m.add_atom('C')               # add atom C using methods inherited from MoleculeContainer
m.add_atom('N')               # add atom N
m.add_bond(1, 2, 1)           # add bond

m.atom(2)['mark'] = 1  # set mark on atom. In this example dictinary setitem supported but update not.

In [ ]:
m.atom(2).mark # one can return mark

# 2. Signatures and duplicates selection

## 2.1. Molecule Signatures
*MoleculeContainer* has methods for unique molecule signature generation.
Signature is SMILES string with explicit bonds notation and canonical atoms ordering. For pyroles, signatures does not comply with the SMILES rules.

For signature generation one need to call `str` function on MoleculeContainer object.  
Fixed length hash of signature could be retrieved by calling `bytes` function on molecule (correspond to SHA 512 bitstring).

Order of atoms calculated by Morgan-like algorithm. On initial state for each atoms it's integer code calculated based on its type. All bonds incident to atoms also coded as integers and stored in sorted tuple. Atom code and tuple of it's bonds used for ordering and similar atoms detecting. Ordered atoms rank is replaced with prime numbers from a prime number lookup table. Atoms of the same type with the same bonds types incident to it have equal prime numbers.

Prime numbers codes found are used in Morgan algorithm cycle.

On each loop for each atom square of its prime number is multiplied to neighboring atoms prime numbers, observed numbers for atoms are ranked and prime numbers are again assigned. Loop is repeated until all atoms will be unique or number of unique atoms will not change in 3 subsequent loops.

In [ ]:
ms2 = str(m2)  # get and print signature
print(ms2)  
# or 
print(m2)

hms2 = bytes(m2)  # get sha512 hash of signature as bytes-string

String formatting is supported that is useful for reporting

In [ ]:
print(f'f string {m2}')  # use signature in string formatting
print('C-style string %s' % m2)
print('format method {}'.format(m2))

Number of neighbors and hybridization could be added to signature. Note that in this case they are not readable as SMILES.

For MoleculeContainer and ReactionContainer query marks are not included in signatures and not printed by `str` and `print` function. However they could be printed in the following way:  

In [ ]:
m2.reset_query_marks() # calculate hybridization and number of neighbors
print(f'{m2:hn}')  # get signatures with hybridization and neighbors data
print('{:h}'.format(m2))  # get signature with hybridization only data
# h - hybridization marks, n- neighbors marks
format(m2, 'n') # include only number of neighbors in signature
print(m2)   # notice that neighbors and hybridization are hided, since signatures for molecules does not contain this information.

Atoms in the signature are represented in the following way: \[*element_symbol*;*hn*;*charge*;*multiplicity* (if not None)\]. h mean hybridization, n - number of neighbors. Notation for hybridization is the following:

* s - all bonds of atom are single
* d - atom has one double bond and others are single
* t - atom has one triple or two double bonds and other are single
* a - atom is in aromatic ring

Examples:
s1 - atom has s hybridization and one neighbor  
d3 - atom has d hybridization and 3 neighbors  

Signatures for QueryContainer and QueryCGRContainer include query marks and are printed by `str` and `print` function. 

Signatures of projections also supported.

In [ ]:
f'{proj:h}'

#### Molecules comparable and hashable 
Comparison of MoleculeContainer is based on its signatures. Moreover, since strings in Python are hashable, MoleculeContaier also hashable.

NOTE: MoleculeContainer can be changed. This can lead to unobvious behavior of the sets and dictionaries in which these molecules were placed before the change. Avoid changing molecules (standardize, aromatize, hydrogens and atoms/bonds changes) placed inside sets and dictionaries.

In [ ]:
m1 != m2 # different molecules

In [ ]:
m7 == m11 # copy of the same molecule

In [ ]:
m7 is m11  # this is not same objects!

In [ ]:
benzene == proj_copy   # projection extracted benzene structure from molecule and then was transformed in MoleculeContainer

In [ ]:
# Simplest way to exclude duplicated structures
len({m1, m2, m7, m11}) == 3 # create set of unique molecules. Only 3 of them were different. 

## 2.2. Reaction signatures
ReactionContainer have its signature. Signature is SMIRKS string in which molecules of reactants, reagents, products presented in canonical order.

API is the same as for molecules

In [ ]:
str(r1)

## 2.3. CGR signature
CGRContainer have its signature. Signatures is SMIRKS-like strings where atoms in reactants and products has same order and split by >> symbol

In [ ]:
str(cgr2)

If one align left- and right-hand side of signature, he will see bond order changes.

C-C-\[O-\].C(=O)(-O)-C(=O)(-O).O-C-C  
C-C-O-C(=O)(.O)-C(=O)(.O)-O-C-C


# 3. Structure standardization

## 3.1. Molecules

MoleculeContainer has `standardize` and `aromatize` methods.

Method `aromatize` transforms Kekule representation of rings into aromatized

Method `standardize` applies functional group standardization rules to molecules. The following rules are implemented (corresponding SMARTS are given):

    • Aromatic N-Oxide 	[#7;a:1]=[O:2]>>[#7+:1]-[#8-:2]
    • Azide 			[#7;A;X2-:1][N;X2+:2]#[N;X1:3]>>[#7:1]=[N+:2]=[#7-:3]
    • Diazo  			[#6;X3-:1][N;X2+:2]#[N;X1:3]>>[#6;A:1]=[N+:2]=[#7-:3]
    • Diazonium  		[#6]-[#7:1]=[#7+:2]>>[#6][N+:1]#[N:2]
    • Iminium  		[#6;X3+:1]-[#7;X3:2]>>[#6;A:1]=[#7+:2]
    • Isocyanate  		[#7+:1][#6;A-:2]=[O:3]>>[#7:1]=[C:2]=[O:3]
    • Nitrilium  		[#6;A;X2+:1]=[#7;X2:2]>>[C:1]#[N+:2]
    • Nitro  			[O:3]=[N:1]=[O:2]>>[#8-:2]-[#7+:1]=[O:3]
    • Nitrone Nitronate 	[#6;A]=[N:1]=[O:2]>>[#8-:2]-[#7+:1]=[#6;A]
    • Nitroso  		[#6]-[#7H2+:1]-[#8;X1-:2]>>[#6]-[#7:1]=[O:2]
    • Phosphonic  		[#6][P+:1]([#8;X2])([#8;X2])[#8-:2]>>[#6][P:1]([#8])([#8])=[O:2]
    • Phosphonium Ylide  	[#6][P-:1]([#6])([#6])[#6+:2]>>[#6][P:1]([#6])([#6])=[#6;A:2]
    • Selenite  		[#8;X2][Se+:1]([#8;X2])[#8-:2]>>[#8][Se:1]([#8])=[O:2]
    • Silicate  		[#8;X2]-[#14+:1](-[#8;X2])-[#8-:2]>>[#8]-[#14:1](-[#8])=[O:2]
    • Sulfine  		[#6]-[#6](-[#6])=[S+:1][#8-:2]>>[#6]-[#6](-[#6])=[S:1]=[O:2]
    • Sulfon  			[#6][S;X3+:1]([#6])[#8-:2]>>[#6][S:1]([#6])=[O:2]
    • Sulfonium Ylide  	[#6][S-:1]([#6])[#6+:2]>>[#6][S:1]([#6])=[#6;A:2]
    • Sulfoxide  		[#6][S+:1]([#6])([#8-:2])=O>>[#6][S:1]([#6])(=[O:2])=O
    • Sulfoxonium Ylide  	[#6][S+:1]([#6])([#8-:2])=[#6;A]>>[#6][S:1]([#6])(=[#6;A])=[O:2]
    • Tertiary N-Oxide  	[#6]-[#7;X4:1]=[O:2]>>[#6]-[#7+:1]-[#8-:2]


In [ ]:
m12 # molecule with kekulized ring

In [ ]:
m12.aromatize() # aromatizes and returns number of transformed rings

In [ ]:
m12 # cleaned structure. Cache is flushed automatically

In [ ]:
m12.standardize()  # apply standardization. Returns number of transformed groups

In [ ]:
m12

Molecules has `explicify_hydrogens` and `implicify_hydrogens` methods to handle hydrogens.

This methods is used to add or remove hydrogens in molecule.

Note that currently for pyrole-like molecules implicit hydrogens atoms are calculated incorrectly  

In [ ]:
m1.explicify_hydrogens() # return number of added hydrogens

In [ ]:
m1 # for added hydrogen atoms coordinates are not calculated. Thus, it looks like hydrogen has the same position on image

In [ ]:
m1.implicify_hydrogens() # return number of removed Hs

In [ ]:
m1

CGRtools has experimental algorithm for 2d geometry calcultaion. It works fine only for small molecules.
Algorithm requires `numpy` and `scipy` packages

In [ ]:
m1.explicify_hydrogens() # add explicit hydrogens
m1.calculate2d() # experimental force field-based 2d geometry calculation.
m1

## 3.2. Reactions standardization
ReactionContainer has `standardize`, `aromatize`, `explicify_hydrogens` and `implicify_hydrogens` methods that can be applied to reactions. In this case they are applied to all molecules in reaction.

In [ ]:
reactions[2]

In [ ]:
reactions[2].standardize()
reactions[2].explicify_hydrogens()
reactions[2]

# 4. Isomorphism

## 4.1. Molecules Isomorphism
CGRtools has simple substructure/structure isomorphism API. In backend VF2 algorithm from `NetworkX` library is used. 

Note, that atoms are matched in subgraph isomorphism only if they have same charge/multiplicity and isotope options.

In [ ]:
m7

In [ ]:
m8

In [ ]:
benzene.standardize()
benzene

In [ ]:
# isomorphism operations
print(benzene < m7)  # benzene is substructure of m7
print(benzene > m7)  # benzene is not superstructure of m7
print(benzene <= m7) # benzene is substructure/or same structure of m7
print(benzene >= m7) # benzene is not superstructure/or same structure of m7
print(benzene < m8) # benzene is not substructure of m8. it's equal
print(benzene <= m8)

In [ ]:
m5

In [ ]:
m6

Mappings of substructure to structure can be returned using `substructure.get_substructure_mapping(structure, limit=1)` method. Argument *limit* is the number of mappings that one wants to be returned, *limit=0* means to return all possible mappings. Method acts as generator.

To get mapping upon structure search `structure1.get_mapping(structure2)` method was developed. It returns only one possible mapping of all atoms for two isomorphic molecules. This functionality was developed to reorder atoms of two MoleculeContainers in the same order (the dictionary that is given by this method could be directly fed to `remap` function, see above) for some reaction handling issues. If molecules are isomorphic it works faster than `get_substructure_mapping`.

In [ ]:
m5.get_substructure_mapping(m6)  # mapping of m5 substructure into m2 superstructure

In [ ]:
for m in m5.get_substructure_mapping(m6, limit=0):  # iterate over all possible substructure mappings
    print(m)

In [ ]:
benzene.get_mapping(m8)  # mapping of benzene into m8 - also benzene.

## 4.2. Reactions
ReactionContainers do not support isomorphism due to ambiguity. But molecules in reaction can be matched.

In [ ]:
try:            # it is not possible to match molecule and reaction. Error is returned
    m6 < r1
except TypeError:
    print(format_exc())

In [ ]:
r1.products[0] # see structure in products

In [ ]:
m6 # substructure used. One can see, they should not match

In [ ]:
any(m6 < m for m in r1.products) # check if any molecule from product side has m6 as substructure

## 4.3 CGR
Substructure search is possible with CGRContainer. API is the same as for molecules.

Matching CGR into CGR and molecule into CGR is possible. **Note that only conventional bonds in CGR could match moleculear bonds.** 

Equal atoms in isomorphism is atoms with same charge/multiplicity and isotope numbers in reactant and product states

In [ ]:
decomposed1 = preparer.decompose(cgr1) # let's have a look at reaction corresponding to cgr1
decomposed1

In [ ]:
m8 # this's the substructure we are looking for

In [ ]:
m8 < cgr1

In [ ]:
cgr1 <= cgr1

## 4.4 Queries

In [ ]:
# to use QueryContainers neighbors and hybridization for molecules need to be calculated
m9.reset_query_marks()
m10.reset_query_marks()

In [ ]:
m9 # acid

In [ ]:
m10 # ether

In [ ]:
carb

In [ ]:
print('m9:', f'{m9:hn}') # all labels were calculated
print('m10:', f'{m10:hn}')
print('carb:', f'{carb:hn}') # notice that one of oxygen atom has 2 neighbors. Only ester could fit this restriction.

Molecules isomorphism don't take into account neighbors and hybridization

In [ ]:
carb < m9 # carb currently is molecule projection. It fit this molecule as well.

In [ ]:
carb < m10 # carb is a substructure of m10 

One need to convert molecule (or it's projection) into QueryContainer object. In this case number of neighbors and hybridization data will be taken into account upon substructure search.

API of isomorphism is the same.

In [ ]:
q = QueryContainer(carb)  # convert molecule into query
print(q)     # now one can see that in signature of QueryContainer. See that one of oxygen has 2 neighbors.

In [ ]:
q < m9 # now neighbors and hybridization are taken into account.  

Acid m9 has hydroxyl group with one non-hydrogen neighbor. Our query requires existence of one oxygen atom with two non-hydrogen neighbors.


In [ ]:
q < m10 # ester matches to query.

In [ ]:
m2.reset_query_marks()
m2

In [ ]:
q < m2 # this molecule does q as substructure as well. It is acid.

# 5. Transformation rules extraction
CGRtools can be used to generate molecules and reactions based on a given transformation rule.

**How to extract transformation rule**

In [ ]:
cgr1.center_atoms # list of atom numbers of reaction center. If several centers exist they will also be added to this list.

In [ ]:
cgr1.center_bonds # list of dynamic bonds as tuples of adjacent atom numbers

In [ ]:
cgr1.centers_list # list of lists of atom numbers belonging to each reaction center. 
                  # Distant reaction centers will be split into separate lists

In [ ]:
rc1 = cgr1.substructure([13, 7]) # get reaction center from CGR
format(rc1, 'hn')               # Notice that query marks are set.

rc1 is phenol reduction, phenol is transformed into unsubstituted benzene:

* \[C;a3;\]-aromatic carbon with 3 neighbors
* \[O;s1;\]-oxygen with 1 neighbor
* \[C;a2;\]-carbon with 2 neighbors
* \[O;s0;\]-oxygen without neighbors (water). It probably appears since initial reaction was unbalanced.

In [ ]:
rule = QueryCGRContainer(rc1) # transform reaction into query to take query 

In [ ]:
print(rule) # all query marks are on their place. Without them generation will be too unrestrictive. 
            # If needed CGRtools could be used to include atomic environment, etc...

# 6. Reactor

*Reactor* objects stores single transformation and can apply it to molecules or CGRs.

Transformations is ReactionContainer object which in reactant side consist of query for matching group and in product side patch for updating matched atoms and bonds 

In [ ]:
from CGRtools import CGRreactor   # import of Reactor
from CGRtools.containers import * # import of required objects

## 6.1. Products generation
Reactor works similar to ChemAxon Reactions enumeration.

Example here presents application of it to create esters from acids.

First we need to construct carboxy group matcher query. Then, ether group need to be specified. 

Atom numbers in query and patch should be mapped to each other. The same atoms should have same numbers.

In [ ]:
acid = QueryContainer()                       # this query matches acids. Use construction possibilities.
acid.add_atom({'element': 'C', 'neighbors': 3})   # add carboxyl carbon. Hybridization is irrelevant here
acid.add_atom({'element': 'O', 'neighbors': 1})   # add hydroxyl oxygen. Hybridization is irrelevant here 
acid.add_atom('O')                                # add carbonyl oxygen. Number of neighbors is irrelevant here.
acid.add_bond(1, 2, 1) # create single bond between carbon and hydroxyl oxygen
acid.add_bond(1, 3, 2) # create double bond
print(acid)

In [ ]:
methyl_ester = MoleculeContainer()  # create patch - how acrboxyl group should be changed. We write methylated group
methyl_ester.add_atom('C', 1) # second argument is predefined atom mapping. Notice that mapping corresponds...  
methyl_ester.add_atom('O', 2) # ... to order in already created acid group. Atom 2 is released water.
methyl_ester.add_atom('O', 4)
methyl_ester.add_atom('O', 3)
methyl_ester.add_atom('C', 5)
methyl_ester.add_bond(1, 4, 1)
methyl_ester.add_bond(1, 3, 2)
methyl_ester.add_bond(4, 5, 1)
# No bond between atom 1 and atom 2. This bond will be broken. 
methyl_ester.calculate2d()
methyl_ester

In [ ]:
m6.reset_query_marks() # required for correct matching
m6 # acid

In [ ]:
template = ReactionContainer([acid], [methyl_ester]) # merge query and patch in template, which is ReactionContainer
reactor = CGRreactor(template)                        # Reactor is initialized
reacted_acid = reactor(m6)                            # application of Reactor to molecule

In [ ]:
reacted_acid.calculate2d(scale=2) # calculate coordinates
reacted_acid       # desired methylated ester have been generated

One can notice presence of separate oxygen (water) and ester group.

The second group can substituted by calling reactor on observed product.

In [ ]:
reacted_acid.reset_query_marks() # this is new molecule and query marks need to be set
second_stage = reactor(reacted_acid) # apply transformation on product of previous transformation
second_stage.calculate2d(scale=2) #  recalculate coordinates for correct drawing
second_stage

*second_stage* has 3 components in a single MoleculeContainer object. We can split it into individual molecules and place all molecules into ReactionContainer object. Since in CGRtools atom-to-atom mapping corresponds to numbering of atoms in molecules, the resulting product has AAM according to the rule applied. Thus, reaction has correct AAM and nothing special should be made to keep or find it.

In [ ]:
products = second_stage.split() # split product into individual molecules
react = ReactionContainer([m6], products) # unite reagent and product into reaction. 
react.fix_positions()
react

For multicomponent reactions one can merge molecules of reactants into single MoleculeContainer object and apply reactor on it.

It is possible to generate all available products in case that molecule has several groups matching the query.

In [ ]:
m6copy = m6.copy() # let's try to use molecule with several groups mathcing query
m6copy.atom(5).isotope = 13 # isotope mark is added to see the difference in products
m6copy.reset_query_marks() # query marks need to be calculated
enums = set()              # the set enums is used to select structurally diverse products
for m in reactor(m6copy, limit=0): # limit=0 is enumeration of all possible products by reactor
    print(m)                       # print signatures for observed molecules. Notice presence of water as component of product
    m.calculate2d(scale=2)         # recalculate coordinates
    enums.update(m.split())        # split product into separate molecules
enums = list(enums)                # set of all resulting molecules

In [ ]:
m6copy

Let's have a look at molecules in set:

In [ ]:
enums[0]

In [ ]:
enums[1]

In [ ]:
enums[2]

## 6.2. MetaReactions (reactions on CGRs).
Reactor could be applied to CGR to introduce changes into reaction. 

### 6.2.1. Example of atom-to-atom mapping fixing. 

In [ ]:
reactions[1] # reaction under study

In [ ]:
cgr = ~reactions[1] # generate reaction CGR
print(cgr)

In [ ]:
cgr.centers_list # reaction has two reaction centers. [10,11,12] - pseudo reaction appeared due to AAM error

reaction has AAM error in nitro-group

![error.png](error.png)

Lets try to use Reactor for AAM fixing

In [ ]:
nitro = QueryCGRContainer() # construct query for invalid reaction center - CGR of wrongly mapped nitro-group
nitro.add_atom({'element': 'N', 'charge': 1, 'p_charge': 1}) # atom 1
nitro.add_atom({'element': 'O', 'charge': 0, 'p_charge': -1}) # atom 2. Notice that due to AAM error charge was changed
nitro.add_atom({'element': 'O', 'charge': -1, 'p_charge': 0}) # atom 3. Notice that due to AAM error charge was changed
nitro.add_atom('C') # atom 4

nitro.add_bond(1, 2, {'order': 2, 'p_order': 1}) # bond between atoms 1 and 2. Due to AAM error bond is dynamic ('2>1' type) 
nitro.add_bond(1, 3, {'order': 1, 'p_order': 2}) # bond between atoms 1 and 3. Due to AAM error bond is dynamic ('1>2' type) 
nitro.add_bond(1, 4, 1) # ordinary bond
print(nitro)
# this query matches reaction center in CGR appeared due to AAM error.


In [ ]:
nitro < cgr # query matches CGR of reaction with error.

In [ ]:
valid_nitro = MoleculeContainer() # construct nitro group without dynamic atoms. Notice that atom order should correspond object nitro
valid_nitro.add_atom({'element': 'N', 'charge': 1}) # ordinary N atom
valid_nitro.add_atom({'element': 'O', 'charge': -1}) # ordinary negatively charged oxygen atom
valid_nitro.add_atom('O')                            # ordinary oxygen atom

valid_nitro.add_bond(1, 2, 1) # ordinary single bond
valid_nitro.add_bond(1, 3, 2) # ordinary double bond
print(valid_nitro)

In [ ]:
valid_nitro.calculate2d()
valid_nitro # this is correct representation of group.

Now time to prepare and apply **Template** to CGR based on reaction with incorrect AAM.

Template is Reaction container with query in reactants and patch in products

In [ ]:
template = ReactionContainer([nitro], [valid_nitro]) # template shows how wrong part of CGR is transformed into correct one.
print(template) # notice complex structure of query: CGR signature is given in braces, then >> and molecule signature

`Reactor` class accept single template. Existence of dynamic bond in it is not a problem.


In [ ]:
reactor = CGRreactor(template)

`Reactor` object is callable and accept as argument molecule or CGR.

NOTE: `fixed` is new CGR object

In [ ]:
fixed = reactor(cgr) # fix CGR

CGRreactor returns None if template could not be applied, otherwise patched structure is returned.

In [ ]:
print(fixed)

`C-C(-I).O-C:1:C:C:C(:C:C:1)-[N+](-[O-])=O`  
`C-C(.I)-O-C:1:C:C:C(:C:C:1)-[N+](-[O-])=O`

One can see that nitro group has no dynamic bonds any more. CGR corresponds only to substitution.

In [ ]:
fixed.centers_list # reaction center appeared due to AAM error before does not exist. Only 1 reaction center is found

Here is depiction of observed CGR (external software was used). Notice absence of wrong reaction center.
![cgr3.png](cgr3.png)

### 6.3.2 Reaction transformation
Example of E2 to SN2 transformation.

E2 and SN2 are concurrent reactions.
We can easily change reaction center of E2 reaction to SN2. It could be achieved by substitution of reaction center corresponding to double bond formation in E2 reaction by the one corresponding to formation of new single bond with base as in SN2.

In [ ]:
from CGRtools import CGRreactor, CGRpreparer
from CGRtools.containers import QueryCGRContainer, ReactionContainer
from CGRtools.files import MRVread, SDFwrite
from io import StringIO

In [ ]:
e2 = next(MRVread('e2.mrv')) # read E2 reaction from ChemAxon MRV file
e2

In [ ]:
# create CGR query for E2 reaction side
e2query = QueryCGRContainer() 
e2query.add_atom('C', 1) # create carbon with mapping number 1
e2query.add_atom('C', 2) # create carbon with mapping number 2
# addition of any halogen atom
e2query.add_atom({'element': ['I', 'Cl', 'Br'], 'neighbors': 1, 'p_neighbors': 0, 'charge': 0, 'p_charge': -1}, 3)
# addition of OH-, RO-, SH-, RS- groups
e2query.add_atom({'element': ['O', 'S'], 'neighbors': [0, 1], 'p_neighbors': [0, 1], 'charge': -1, 'p_charge': 0}, 4)

e2query.add_bond(1, 2, {'order': 1, 'p_order': 2}) # bond between two carbon corresponds to formation of double from single
e2query.add_bond(1, 3, {'order': 1, 'p_order': None}) # bond between carbon and halogen breaks in E2 reaction
print(e2query) # it is CGR of E2 reaction center

In [ ]:
e2_cgr = ~ e2 # compose reaction into CGR
e2_cgr.reset_query_marks() # prepare to isomorphism

CGR is the following (depicted by external software)
![cgr4.png](cgr4.png)

In [ ]:
e2query < e2_cgr # E2 CGR pattern works!

In [ ]:
# create patch creating SN2 reaction. Notice that ordering of atoms correspond to that of E2 CGR query
sn2patch = QueryCGRContainer()
sn2patch.add_atom({}, 1) # save atom unchanged. We don't specify atom type since it will be taken from E2 query
sn2patch.add_atom({}, 2) # it is central atom. We don't specify atom type since it will be taken from E2 query
sn2patch.add_atom({'charge': 0, 'p_charge': -1}, 3) # elements list with same order of elements [I, Cl, Br] could be used as well
sn2patch.add_atom({'charge': -1, 'p_charge': 0}, 4)

sn2patch.add_bond(1, 2, {'order': 1, 'p_order': 1}) # set carbon - carbon single bond that is unchanged in SN2 reaction
sn2patch.add_bond(1, 3, {'order': 1, 'p_order': None}) # this bond is broken in SN2 reaction
sn2patch.add_bond(1, 4, {'order': None, 'p_order': 1}) # it corresponds to formation of bond O(S)-C bond in SN2 reaction

In [ ]:
reactor = CGRreactor(ReactionContainer([e2query], [sn2patch])) # create template and pass it to Reactor
sn2_cgr = reactor(e2_cgr) # apply Reactor on E2 reaction

In [ ]:
print(sn2_cgr)

It is depiction of CGR produced by Reactor (external software is used)
![cgr5.png](cgr5.png)

In [ ]:
# decompose CGR into reaction
preparer = CGRpreparer() 
sn2 = preparer.decompose(sn2_cgr)

In [ ]:
sn2.calculate2d()
sn2 # reaction has the same reagents like E2 above, but products correspond to SN2 reaction

# 7. Input-output operations
*CGRtools.files* subpackage contains file readers and writers classes.

## 7.1. MDL RDF reader

**RDFread** class can be used for RDF files reading.
Instance of this class is file-like object which support **iteration**, has a method **read()** for parsing all data and **context manager**.


### 7.1.1. Read file from disk

In [ ]:
from CGRtools.files import * # import all available readers and writers

with RDFread('example.rdf') as f:
    first = next(f)  # get first reaction using generator
    data = f.read()  # read remaining reactions to list of ReactionContainers

data = []
with RDFread('example.rdf') as f:
    for r in f:  # looping is supported. Useful for large files.
        data.append(r)

with RDFread('example.rdf') as f:
    data = [r for r in f]  # list comprehensions application. Result is equivalent to f.read() 

#### OOP-stype Pathlib supported

In [ ]:
from pathlib import Path

with RDFread(Path('example.rdf')) as r: # OOP style call
    r = next(r)

#### opened files supported
RDF file should be opened in text mode

In [ ]:
with open('example.rdf') as f, RDFread(f) as r:
    r = next(r) # OOP style application

### 7.1.2. Transparent loading from archives and network
Readers designed transparently support any type of data sources. 

Page https://cimm.kpfu.ru/seafile/f/aeaca685e3854ae2bbad/?dl=1 returns RDF file.

Data sources should be file-like objects.

In [ ]:
from requests import get
from io import StringIO

# get function return requested URL which has attribute text. 
# in example this text is whole RDF stored in single string.
# RDFread does not support parsing of strings, but one can emulate files with data 
# instead of strings by using io.StringIO
with StringIO(get('https://cimm.kpfu.ru/seafile/f/aeaca685e3854ae2bbad/?dl=1').text) as f, RDFread(f) as r:
    r = next(r)
    print(r, 'StringIO downloaded from network data')

# python support gzipped data. This example shows how to work with compressed 
# data directly without decompressing them to disk.
from gzip import open as gzip_open
with gzip_open('example.rdf.gz', 'rt') as f, RDFread(f) as r:
    r = next(r)
    print(r, 'gzipped file')

# zip-files also supported out of the box 
# zipped files can be opened only in binary mode. io.TextIOWrapper can be used for transparent decoding them into text
from zipfile import ZipFile
from io import TextIOWrapper
with ZipFile('example.zip') as z, z.open('example.rdf') as c:
    with TextIOWrapper(c) as f, RDFread(f) as r:
        r = next(r)
        print(r, 'zip archive')

# tar-file reading example
from tarfile import open as tar_open
from io import TextIOWrapper
with tar_open('example.tar.gz') as t:
    c = t.extractfile('example.rdf')
    with TextIOWrapper(c) as f, RDFread(f) as r:
        r = next(r)
        print(r, 'gzipped tar archive')

## 7.2. Other Readers
* SDFread - MOL, SDF files reader (versions v2000, v3000 are supported)
* MRVread - ChemAxon MRV files reader (lxml parser is used)
* SMILESread - SMILES strings files reader (coho backend used). Every row should start with new SMILES
* INCHIread - INCHI strings files reader (INCHI trust backend used). Every row should start with new InChI

All files except MRV should be opened in **text-mode**  
MRV requires binary mode `open('/path/to/data.mrv', 'rb')`

In [ ]:
with MRVread(open('example.mrv', 'rb')) as f:
    mrv = next(f)
mrv

## 7.3. File writers
Export in following file formats is supported:
* RDFwrite (v2000) - molecules and reactions  export in RDF format
* SDFwrite (v2000) - molecules and CGR export in SDF format
* MRVwrite - molecules and reactions export in MRV format

Writers has the same API as readers. All writers work with text-files
Writers has `write` method which accepts as argument single reaction, molecule or CGR object

In [ ]:
with RDFwrite('out.rdf') as f: # context manager supported
    for r in data:
        f.write(r)
# file out.rdf will be overriden

In [ ]:
f = RDFwrite('out.rdf') # ongoing writing into a single file
for r in data:
    f.write(r)

f.write(r1)

f.close() # close file. Flushes Python writer buffers.

## 7.4. CGR can be stored in MDL SDF and loaded from.

White-paper with SDF-CGR specification is described in manusript Supporting Materials.

In [ ]:
from CGRtools.files import *
from io import StringIO

with StringIO() as f,  SDFwrite(f) as w:
    w.write(cgr2) # file writing in SDF format
    mdl = f.getvalue() # get formatted file to print out
print(mdl) # It is how CGR looks like. 
# Notice that most of field are conventional MOL fields, S-queries are used for dynamic bond and atom specification

In [ ]:
with StringIO(mdl) as f,  SDFread(f) as r: # import SDF file with CGR
    cgr3 = next(r)
print(cgr3)
print(type(cgr3))

## 7.5. Pickle support

CGRtools containers fully support pickle dumping and loading.

Moreover backward compatability is declared since 3.0.  
Any new version of library can load dumps created with older version.

Pickle dumps are more compact than MDL files and could be used as temporal storage.

In [ ]:
from pickle import loads, dumps

In [ ]:
loads(dumps(r1)) # load reaction from Pickle dump